<a href="https://colab.research.google.com/github/daleelenteny/NWI_Workflow/blob/main/edatt_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import arcpy
from arcgis.gis import GIS
import arcgis
import arcpy
import pandas as pd
import string
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureLayerCollection
from arcgis.mapping import WebMap
import arcgis.geometry
from arcgis.geometry import Point

pd.options.display.float_format = '{:,}'.format

agol_url = "https://monroemi.maps.arcgis.com/home/index.html"
portal_url = "https://gis.mocogeo.org/portal/home/"
portal_id_url = "https://gis.mocogeo.org/portal/home/item.html?id="
agol_id_url = "https://monroemi.maps.arcgis.com/home/item.html?id="
agol_name = "username"
portal_name = "password"
agol_password = "username"
portal_password = "password"

# Log in to AGOL and Portal
agol = GIS(url=agol_url, username=agol_name, password=agol_password)
portal = GIS(url=portal_url, username=portal_name, password=portal_password, profile = 'Dale', verify_cert = False)

# Dependencies
block_groups = r"https://gis.mocogeo.org/server/rest/services/Hosted/Education_Attainment_Data_Tool_Geodatabase_FLC_/FeatureServer/1"
parcels = r"https://gis.mocogeo.org/server/rest/services/Hosted/Education_Attainment_Data_Tool_Geodatabase_FLC_/FeatureServer/2"
ea_mp = portal.content.get("63bdb898e2c2437097e715d583a2b4dd")

def ScriptTool(parcel_input, search_distances):
# Identify parcel and block group layers
    flc = portal.content.search("7bdaa7ac55e5404490f16d2ae9744bcf")[0]
    for f in flc.layers:
        if f.properties.name == "Parcels":
            parcels = f.url
        if f.properties.name == "Block_Groups":
            block_groups = f.url
### Part One: Create integer list (radii) of search distance inputs
    arcpy.SetProgressor("step", "Creating list of search distances...", 0, 7, 1)
    arcpy.AddMessage("Creating list of search distances...")
    radii = []
    for sd_int in search_distances.split(";"):
        sd_int = int(str(sd_int))
        radii.append(sd_int)

### Part Two: Create polygon list of buffers around the specified point
    arcpy.SetProgressor("step", "Creating a polygon for each search area...", 0, 7, 1)
    arcpy.AddMessage("Creating a polygon for each search area")
    buffers = []
    polygon_jsons = []
    buffer_jsons = []
    sr = arcpy.Describe(parcels).spatialReference
    with arcpy.da.SearchCursor(parcels, ["x", "y", "parcelid"]) as cursor:
        for row in cursor:
            if row[2] == parcel_input:
                x = row[0]
                y = row[1]
                pt = arcpy.Point(x, y)
                pg = arcpy.PointGeometry(pt, sr)
                for r in radii:
                    meters = r * 1609.34                                                                           #################CLIPPIN####################
                    poly = pg.buffer(meters)
                    poly
                    buffers.append(poly)
                    polygon_jsons.append(poly.JSON)

    for b in buffers:
        buffer_jsons.append(b.JSON)

### Part Three: Create list of block group selections
    arcpy.SetProgressor("step", "Selecting census block group data...", 0, 7, 1)
    arcpy.AddMessage("Selecting census block group data...")
    bg_areas = []
    for polygon in buffers:
        bg_selection, bg_multiple, count = arcpy.management.SelectLayerByLocation(block_groups, "INTERSECT", polygon)
        #print(bg_selection, bg_multiple, count)
        bg_areas.append(bg_selection)                                                                            ######## At this point an error has occured with the first bg selection

### Part Four: Create sum_fields list variable
    arcpy.SetProgressor("Calculating data...", 0, 7, 1)
    arcpy.AddMessage("Calculating data...")
    sum_fields = []
    block_groups = flc.layers[1]
    for field in block_groups.properties.fields:
        if field.editable == True:
            field.type = "esriFieldTypeInteger"
            sum_fields.append(field.name)
    sum_fields.remove("fips")

### Part Five: Create field dictionary that will be used to populate dataframe. All values will be 0 at this point
    field_dict = {}
    col_names = []
    for field in sum_fields:
        field_dict[field] = 0
        col_names.append(field)
    # I think these will be important?
    row_count = 0
    tract_count = 0
    key_list = []
    value_list = []

### Part Six: Algorithm to populate dictionary
    df_dict = {}
    for bg_selection in bg_areas:
        tract_count += 1
        result = arcpy.GetCount_management(bg_selection)
        ftr_count = int(result.getOutput(0))
        for fd_key, fd_value in field_dict.items():
            with arcpy.da.SearchCursor(bg_selection, fd_key) as rows:
                for row in rows:
                    row_count += 1
                    row = str(row)
                    row = row.replace(",", "").replace("(", "").replace(")", "").replace("'", "")
                    row = int(row)
                    field_dict[fd_key] += row
        for key, value in field_dict.items():
            if key not in key_list:
                key_list.append(key)
                df_dict[key] = []
            if key in key_list and df_dict[key] == ():
                df_dict[key] = value
            if key in key_list and df_dict[key] != ():
                df_dict[key].append(value)

### Part Seven: Dictionary into dataframe
    df = pd.DataFrame.from_dict(df_dict)
    df.insert(0, "Search_Radius_in_Miles", radii)
    col_names.remove("total_population")
    merge_dict = {}
    for col in col_names:
        merge_dict[col] = "perc_{}".format(col)
        loc = df.columns.get_loc(col) + 1
        df.insert(loc, "perc_{}".format(col), 0)
        num = df[col].astype(int)
        den = df["total_population"].astype(float)
        perc = ((num/den)*100)
        perc = perc.round(decimals=2)
        df["perc_{}".format(col)] = perc
        df = df.round(2)

### Part Eight: Dataframe to Spatial Dataframe to feature layer
    arcpy.SetProgressor("step", "Exporting dataframe to feature layer...", 0, 7, 1)
    arcpy.AddMessage("Exporting dataframe to feature layer...")

    # Insert commas
    df = df.applymap('{:,.2f}'.format)                                                                              # This is where I will format the numbers

    # Insert geometry column
    df["SHAPE"] = buffer_jsons
    sdf = GeoAccessor.from_df(df, geometry_column = "SHAPE")
    #sdf.set_index("Search_Radius_in_Miles", inplace = True)
    # Define sdf properties
    title = "Output_Data"
    gis = portal
    tags = "edatt"
    folder = "Education Attainment Outputs"                                                                          # Will sharing this folder be important?
    sanitize_columns = True
    service_name = "sdf"
    # Export to feature layer
    source_sdf_fl = sdf.spatial.to_featurelayer(title, gis, sanitize_columns)
    source_url = source_sdf_fl.url

### Part Nine: Edit field aliases
    arcpy.SetProgressor("step", "Cleaning up data...", 0, 7, 1)
    arcpy.AddMessage("Cleaning up data...")
    # Identify layer to work on
    l = source_sdf_fl.layers[0]
    # This is the key/property I will use
    orig_di = l.properties['drawingInfo']

    # Construct an alias dictionary
    alias_dict = {}
    for field in l.properties.fields:
        if field.editable == True:
            # Phrase-level replacements
            orig_name = field.name
            new_name = orig_name.replace("_s_", "'s ")
            new_name = new_name.replace("_", " ")
            new_name = new_name.replace("perc", "Percentage with")
            new_name = new_name.replace("ged", "GED")
            # Word-level replacements
            phrase = ""
            # Adding in appropriate phrase
            abs_names = ["Percentage", "search", "total", "Adults 25+"]
            if new_name.split(" ")[0] not in abs_names:
                new_name = new_name.replace(new_name, "Adults 25+ with " + new_name)
            # Capitalizing words
            for word in new_name.split(" "):
                if word != "with" and word != "or":
                    word = word.replace(word, word[0].upper() + word[1:])
                if len(phrase) > 0:
                    phrase = phrase + " " + word
                else:
                    phrase = word
            # Define dictionary
            alias_dict[orig_name] = phrase
    # Identify layer to work on
    #l = source_sdf_fl.layers[0]
    # Create an empty list of new fields
    # Iteratively add to it based on properties within the dictionaries
    new_fields = []
    for a in l.properties['fields']:
        if a['editable'] == False:
            new_fields.append(a)
        if a['editable'] == True:
            for key, val in alias_dict.items():
                if a['alias'] == key:
                    a['alias'] = val
                    new_fields.append(a)

### Part Ten: Update drawing info
    # Palette
    dark_blue = [35, 97, 146, 150, 1.0]
    pale_blue = [164, 190, 208, 150]
    gray = [227, 233, 237, 150]
    teal = [128, 224, 214, 150]
    # Preset symbols
    outline = {'color': dark_blue, 'width':'2', 'type':'esriSLS', 'style':'esriSLSLongDash'}
    # Identify layer and start with drawingInfo, work from there
    l = source_sdf_fl.layers[0]
    orig_di = l.properties['drawingInfo']
    # Create a copy of it
    new_di = orig_di
    # Newly define as a simple renderer
    new_di['renderer']['type'] = 'simple'
    new_di['renderer']['symbol'] = {'color':pale_blue, 'type':'esriSFS', 'style':'esriSFSSolid', 'outline':outline}
    # Update definition
    l.manager.update_definition({'drawingInfo':new_di})

### Part Eleven: Add layer to map and configure map
    arcpy.SetProgressor("step", "Adding layer to map...", 0, 7, 1)
    # Add layer to map
    mp = portal.content.get("63bdb898e2c2437097e715d583a2b4dd")
    ea_mp = WebMap(mp)
    for l in ea_mp.layers:
        if l.title != "Education Attainment  Data Tool Geodatabase FLC  - Parcels" and l.title != "Ontario" and l.title != "Monroe County Boundary":
            ea_mp.remove_layer(l)
            ea_mp.update()
    ea_mp.add_layer(source_sdf_fl, options = {'opacity':0.65})
    ea_mp.update()
    # Change the output name
    # This is how I identify the random output name
    output_name = source_sdf_fl.layers[0].properties.name
    # This changes the title of the layer - I just need to find a way to obtain the id so it does this every time
    mp = portal.content.get("63bdb898e2c2437097e715d583a2b4dd")
    ea_mp = WebMap(mp)
    for l in ea_mp.layers:
        if l['title'] == output_name:
            l['title'] = "Output Data"
            # Now this edits the popups
            output_data = l
            output_data.popupInfo['title'] = "Search Radius in Miles: {search_radius_in_miles}"
            for popup in output_data.popupInfo['fieldInfos']:
                if popup['isEditable'] == True:
                    popup['visible'] = True
                else:
                    popup['visible'] = False
                ea_mp.update()

    # Define extent dictionary
    # Assign extent values based on dictionary
    extent_dict = {}
    extent_dict['xmin'] = source_sdf_fl.extent[0][0]
    extent_dict['ymin'] = source_sdf_fl.extent[0][1]
    extent_dict['xmax'] = source_sdf_fl.extent[1][0]
    extent_dict['ymax'] = source_sdf_fl.extent[1][1]
    # Assign extent values based on dictionary
    for k, v in extent_dict.items():
        ea_mp.definition['initialState']['viewpoint']['targetGeometry'][k] = v
    ea_mp.update()

    # Finish
    return radii, buffers, poly, polygon_jsons, buffer_jsons, bg_areas, poly, sum_fields, col_names, field_dict, df_dict, merge_dict, df, sdf, source_sdf_fl, source_url, new_fields, l, output_data, orig_di, ea_mp

# This is used to execute code if the file was run but not imported
if __name__ == '__main__':
    # Tool parameter accessed with GetParameter or GetParameterAsText
    parcel_input = arcpy.GetParameterAsText(0)
    search_distances = arcpy.GetParameterAsText(1)

radii, buffers, poly, polygon_jsons, buffer_jsons, bg_areas, poly, sum_fields, col_names, field_dict, df_dict, merge_dict, df, sdf, source_sdf_fl, source_url, new_fields, l, output_data, orig_di, ea_mp = ScriptTool(parcel_input, search_distances)

# This is used to execute code if the file was run but not imported
if __name__ == '__main__':
    # Tool parameter accessed with GetParameter or GetParameterAsText
    parcel_input = arcpy.GetParameterAsText(0)
    search_distances = arcpy.GetParameterAsText(1)

ScriptTool(parcel_input, search_distances)